<a href="https://colab.research.google.com/github/VavRe/nlp-ut/blob/main/CA5/Fairseq-RAW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
!pip install fairseq
!pip install sacremoses
!pip install tensorboardX

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [33]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:  
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [34]:
import os

In [35]:
#join all paths to a base path 
BASE_PATH = "/content/drive/MyDrive/NLP/CA5"
RAW_DATA_PATH = os.path.join(BASE_PATH, "raw_data")
TRAIN_DATA_PATH = os.path.join(RAW_DATA_PATH, "train")
VALID_DATA_PATH = os.path.join(RAW_DATA_PATH, "valid")
TEST_DATA_PATH = os.path.join(RAW_DATA_PATH, "test")
VOCAB_SIZE = 5000
LEARNING_RATE = 0.0025
LABEL_SMOOTHING = 0.2
ADAM_BETA11 = 0.9
ADAM_BETA22 = 0.98
DROPOUT = 0.25

In [36]:
!mkdir -p ./data_bin

In [37]:
!fairseq-preprocess --source-lang en --target-lang fa \
  --trainpref {TRAIN_DATA_PATH} \
  --validpref {VALID_DATA_PATH} \
  --testpref {TEST_DATA_PATH} \
  --nwordstgt 40000 \
  --nwordssrc 40000 \
  --destdir ./data_bin/


2023-06-08 21:53:18.860348: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-08 21:53:19.757980: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-08 21:53:22 | INFO | fairseq_cli.preprocess | Namespace(no_progress_bar=False, log_interval=100, log_format=None, log_file=None, aim_repo=None, aim_run_hash=None, tensorboard_logdir=None, wandb_project=None, azureml_logging=False, seed=1, cpu=False, tpu=False, bf16=False, memory_efficient_bf16=False, fp16=False, memory_efficient_fp16=False, fp16_no_flatten_grads=False, fp16_init_scale=128, fp16_scale_window=None, fp16_scale_tolerance=0.0, on_cpu_convert_precision=False, min_loss_scale=0.0001, threshold_loss_scale=None, amp=False, amp_batch_retries=2,

In [ ]:
!fairseq-train \
    "./data_bin/" \
    --arch lstm --share-decoder-input-output-embed \
    --optimizer adam --adam-betas '(0.9,0.98)' --clip-norm 0.0 \
    --lr 2.5e-3 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --dropout 0.25 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.2 \
    --max-tokens 4096 \
    --eval-bleu \
    --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
    --eval-bleu-detok moses \
    --eval-bleu-print-samples \
    --best-checkpoint-metric bleu --maximize-best-checkpoint-metric \
    --fp16 --memory-efficient-fp16 \
    --max-epoch 5 \
    --save-dir ./content/drive/MyDrive/raw/checkpoints/ \
    --tensorboard-logdir ./content/drive/MyDrive/raw/logs



2023-06-08 21:53:24.926281: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-08 21:53:25.817811: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-08 21:53:27 | INFO | numexpr.utils | NumExpr defaulting to 2 threads.
2023-06-08 21:53:30 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': './content/drive/MyDrive/raw/logs', 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': True, 'fp16_no_flatten_grads': False, 

In [ ]:
!fairseq-generate \
    "./data_bin" \
    --batch-size 128 \
    --path "./content/drive/MyDrive/raw/checkpoints/checkpoint_best.pt" \
    --beam 5 > "./content/drive/MyDrive/raw/new_eval.txt"

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir "./content/drive/MyDrive/raw/logs" --port=6009